In [1]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [2]:
%env NUMBA_WARNINGS 1
%env NUMBA_NUM_THREADS 12

import ctypes
mkl_rt = ctypes.CDLL('libmkl_rt.so')
print(mkl_rt.mkl_get_max_threads())
mkl_get_max_threads = mkl_rt.mkl_get_max_threads
def mkl_set_num_threads(cores):
    mkl_rt.mkl_set_num_threads(ctypes.byref(ctypes.c_int(cores)))
mkl_set_num_threads(12)
print(mkl_get_max_threads())

env: NUMBA_WARNINGS=1
env: NUMBA_NUM_THREADS=12
16
12


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import os
import pickle as pkl
from telepyth import TelepythClient
import time

from approximate import APPROX
from experiment import relative_errors
from dataset import make_dataset
from visualize import plot

Use token from .telepythrc.


# USPS

In [4]:
approx_types = ['G', 'Gort', 'ROM', 'QMC', 'GQ', 'B']
# halton sequence generator cannot generate large enough sequences for datasets CIFAR100 and LEUKEMIA
# same with GQ method, where the scipy Hermite polynomials roots generator also restricts the max dataset dim.
approx_types_large_d = ['G', 'Gort', 'ROM', 'B']

In [5]:
datasets = ['USPS', 'LETTER', 'Powerplant', 'MNIST', 'CIFAR100', 'LEUKEMIA']

In [10]:
datasets = ['LEUKEMIA'] #['LETTER', 'Powerplant', 'MNIST', 'CIFAR100', 'LEUKEMIA']

In [11]:
kernels = ['Arccos 1', 'RBF', 'Arccos 0']

In [ ]:
%%time
%%telepyth
sample_params = [1, 3, 1, 0, 1, 10, 50]
for name in datasets:
    X, Y, params = make_dataset(name, sample_params)
    errs = {}
    start_deg, max_deg, step, shift, runs, _, _ = params
    if X.shape[1] > 784:
        approx_types = approx_types_large_d
        
    for kernel in kernels:
        errs[kernel] = relative_errors(X, Y, kernel, approx_types, start_deg, max_deg, step, shift, runs)
        
    directory = 'results/%s/' % name
    if not os.path.exists(directory):
        os.makedirs(directory)

    with open(directory + '%s_%r_%r' % (name, kernels, params), 'wb') as f:
        pkl.dump(errs, f)
        
    plot(errs)

  0%|          | 0/4 [00:00<?, ?it/s]

G


 25%|██▌       | 1/4 [00:39<01:57, 39.33s/it]

Gort


 50%|█████     | 2/4 [03:07<02:24, 72.00s/it]

...

B


  0%|          | 0/4 [00:00<?, ?it/s]

G


 25%|██▌       | 1/4 [00:34<01:44, 34.85s/it]

Gort
